In [1]:
import spacy
from spacy import displacy

#### PHẦN 2: Phân tích câu và trực quan hóa

In [2]:
# Tải mô hình tiếng Anh đã cài đặt
# Sử dụng en_core_web_md vì nó chứa các vector từ và cây cú pháp đầy đủ
nlp = spacy.load("en_core_web_md")

# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."

# Phân tích câu với pipeline của spaCy
doc = nlp(text)

# In thông tin cơ bản
for token in doc:
    print(token.text, token.dep_, token.head.text)

The det fox
quick amod fox
brown amod fox
fox nsubj jumps
jumps ROOT jumps
over prep jumps
the det dog
lazy amod dog
dog pobj over
. punct jumps


In [6]:
# trực quan hóa cây phụ thuộc
# Khởi chạy web server để hiển thị cây phụ thuộc

## trick: monkey patch để không lỗi thư viện
import IPython.display
import IPython.core.display

IPython.core.display.display = IPython.display.display
IPython.core.display.HTML = IPython.display.HTML


displacy.render(doc, style="dep", jupyter=True)

# Truy cập: http://127.0.0.1:5000

#### Phần 3: Truy cập các thành phần trong cây phụ thuộc

In [7]:
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)

print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")


TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


#### Phần 4: Duyệt cây phụ thuộc để trích xuất thông tin

In [8]:
# Tìm (Chủ ngữ – Động từ – Tân ngữ) - Bài toán: Tìm chủ ngữ và tân ngữ của một động từ

text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)

for token in doc:
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""

        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text

        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")


Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


In [9]:
# Tìm các tính từ bổ nghĩa cho danh từ
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)

for token in doc:
    if token.pos_ == "NOUN":
        adjectives = []

        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)

        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")


Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi các tính từ: ['warm']


#### Bài 1: Tìm động từ chính (ROOT)

In [10]:
def find_main_verb(doc):
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            return token
    return None

# Test
doc = nlp("The students are studying dependency parsing.")
main_verb = find_main_verb(doc)
print("Main verb:", main_verb)


Main verb: studying


#### Bài 2: Tự trích xuất cụm danh từ (Noun Chunk)

In [13]:
def extract_noun_chunks(doc):
    noun_chunks = []

    for token in doc:
        if token.pos_ == "NOUN":
            chunk_tokens = []

            # Thêm các từ bổ nghĩa phía trước
            for child in token.children:
                if child.dep_ in ("det", "amod", "compound"):
                    chunk_tokens.append(child.text)

            chunk_tokens.append(token.text)

            noun_chunks.append(" ".join(chunk_tokens))

    return noun_chunks

# Test
doc = nlp("The big fluffy cat sat on the wooden table.")
print("Noun Chunks:", extract_noun_chunks(doc))


Noun Chunks: ['The big fluffy cat', 'the wooden table']


#### Bài 3: Tìm đường đi từ token lên ROOT

In [15]:
def get_path_to_root(token):
    path = [token.text]

    while token.head != token:
        token = token.head
        path.append(token.text)

    return path

#  Test
doc = nlp("The cat chased the mouse.")
token = doc[4]  # mouse
print("Path to ROOT:", get_path_to_root(token))


Path to ROOT: ['mouse', 'chased']
